In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [2]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
cid = 
csecret = 

In [4]:
uid='jesjes354'

In [6]:
#https://towardsdatascience.com/playlist-classification-on-spotify-using-knn-and-naive-bayes-classification-35a279b7e255

client_credentials_manager = SpotifyClientCredentials(client_id=cid, 
                                                      client_secret=csecret) 
scope = 'user-top-read'
redirect_uri="https://github.com/jessimk"
try:
    token = util.prompt_for_user_token(uid, scope, 
                                       client_id=cid, 
                                       client_secret=csecret, 
                                       redirect_uri=redirect_uri)  
    sp=spotipy.Spotify(auth= token)
except:
    print('Token is not accesible for ' + uid)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=d6495ca01f17418697b138a6d76425b3&response_type=code&redirect_uri=https%3A%2F%2Fgithub.com%2Fjessimk&scope=user-top-read in your browser


Enter the URL you were redirected to: https://github.com/jessimk?code=AQA9pEu6zUc6x44izbeXKeBkbB4RY1figHKtWf4XUyQp7OgWg2t8zlW83w_IKOkyrgsjK_boJsIqaZiGtonVFLBwEZVNZkxdOoS_CfXUhBh1wRQag9hzkOJFge-VgWod0nbvMBkuSNyvRIvhYJu7kP7fQUoci-sL_JYBXj2nnp11OfnMubWiTmVkjKKq2m4O7eY0g75O28wfQtzkKobx




In [9]:
top3 = sp.current_user_top_tracks(limit=2, offset=0, time_range="short_term")

In [21]:
top3['items'][0]['album']['name']

'Shawn Mendes'

In [29]:
top3['items'][0]['album']['artists'][0]['name']

'Shawn Mendes'

In [23]:
top3['items'][0]['album']['release_date']

'2018-05-25'

In [30]:
top3['items'][0]['name']

'Lost In Japan'

In [33]:
top3['items'][0]['duration_ms']

201200

In [34]:
top3['items'][0]['popularity']

81

In [35]:
top3['items'][0]['id']

'79esEXlqqmq0GPz0xQSZTV'

In [32]:
top3['items'][0]['explicit']

False

In [ ]:
#https://github.com/jessimk/525/blob/master/DSCI_525_lab1_jess354-master/lab1.md
df = pd.DataFrame({"album_name": [], "album_artist": [], "album_release_dt": [], "track_artist": [],
                  "track_name": [], "track_dur_ms": [], "track_popularity": [], "track_id": [],
                    "track_explicit": []})
row = []
df = []
for i,entry in enumerate(tables[0].find_all("td")):
    row.append(entry.text)
    if (i+1)%3 == 0:
        df.append(row)
        row = []

df = pd.DataFrame(df, columns = ["n_titles", "team", "years"])
df